In [ ]:
import os
from util import *
from search import search, _find_gapped_pairs
from agnostic_search import search as agnostic_search, _find_gapped_pairs as agnostic_find_gapped_pairs
from prototype import *
from random import shuffle, randint
from matplotlib import pyplot as plt

tolerance = 2 * AVERAGE_MASS_DIFFERENCE
alphabet = AMINO_MASS_MONO
amin = min(alphabet)
amax = max(alphabet)

In [ ]:
seqs = load_fasta_records_as_str("uniprot_sprot.fasta")
shuffle(seqs)
seqs = seqs[:100]
peps = collapse_second_order_list(map(digest_trypsin,seqs))
peps = list(filter(lambda pep: 'X' not in pep, peps))
specs = list(map(generate_spectrum_and_list_mz,peps))
specs = [np.unique(spec) for spec in specs]

## Timing and Sanity Checks

In [ ]:
from time import time
def duration(fn, specs, *args):
    init_t = time()
    for spec in specs:
        fn(spec,*args)
    return time() - init_t

In [ ]:
duration(old_search_overlap,specs,tolerance,alphabet)

In [ ]:
duration(old_search_overlap_alt,specs,tolerance,alphabet)

In [ ]:
duration(search,specs,"overlap",alphabet,tolerance)

In [ ]:
duration(search,specs,"overlap_alt",alphabet,tolerance)

## Manual Validation

#### Compare Inferred Pivots to True Pivots
for validation on large datasets, `python3 prototype.py test path/to/fasta`

In [ ]:
def manual_check(spec,pep,searchmode="gap-driven"):
    b = get_b_ion_series(pep)
    y = get_y_ion_series(pep)
    true_pivot = np.mean([*b[0:2],*y[-3:-1]])
    predicted_pivot = locate_pivot_point(spec,tolerance,searchmode)
    print("pivot",predicted_pivot)
    print("true pivot",true_pivot)
    print("error",abs(true_pivot - predicted_pivot))
    print("symmetry, expected symmetry",measure_mirror_symmetry(spec,predicted_pivot), (len(spec) - 1)/len(spec))

In [ ]:
i = randint(0,len(specs)-1)
manual_check(specs[i],peps[i])

#### Compare gap-driven and gap-agnostic pair discovery

In [ ]:
def manual_compare(spec):
    pairs_g = collapse_second_order_list([_find_gapped_pairs(spec,tg,tolerance) for tg in alphabet])
    pairs_a = agnostic_find_gapped_pairs(spec,amin,amax,tolerance)
    print("gap-driven pairs",len(pairs_g))
    print("gap-agnostic pairs",len(pairs_a))
    inc = 0
    for p in pairs_g:
        if p in pairs_a:
            inc += 1
    print("% gap-driven pairs present in gap-agnostic pairs",100 * inc / len(pairs_g))

In [ ]:
i = randint(0,len(specs)-1)
manual_compare(specs[i])

#### Peptides of Unsolved Spectra 

In [ ]:
def manual_false_misses(path_fasta,searchmode):
    peps = load_fasta_records_as_str(path_fasta)
    specs = list(map(generate_spectrum_and_list_mz,peps))
    false_misses = []
    for i in range(len(specs)):
        spec = specs[i]
        pep = peps[i]
        predicted_pivot = locate_pivot_point(spec,tolerance,searchmode)
        b = get_b_ion_series(pep)
        y = get_y_ion_series(pep)
        true_pivot = np.mean([*b[0:2],*y[-3:-1]])
        if abs(true_pivot - predicted_pivot) < 0.1 and check_pivot(predicted_pivot,pep,spec):
            false_misses.append((i,pep))
    num_false_misses = len(false_misses)
    return false_misses, num_false_misses, num_false_misses / len(specs)

In [ ]:
manual_false_misses("misses/misses_gap-agnostic_uniprot_sprot.fasta","gap-agnostic")

In [ ]:
manual_false_misses("misses/misses_gap-driven_uniprot_sprot.fasta","gap-driven")

In [ ]:
manual_false_misses("misses/misses_gap-agnostic_uniprot_sprot.fasta","gap-driven")[1:]

In [ ]:
manual_false_misses("misses/misses_gap-driven_uniprot_sprot.fasta","gap-agnostic")[1:]

In [ ]:
gd_miss_peps = load_fasta_records_as_str("misses/misses_gap-driven_uniprot_sprot.fasta")
gd_miss_specs = list(map(generate_spectrum_and_list_mz,gd_miss_peps))
idx = -1

In [ ]:
idx = (idx + 1) % len(gd_miss_specs)
print(idx)
pep = gd_miss_peps[idx]
spec = gd_miss_specs[idx]
print(pep, len(pep))
print(spec)
p = locate_pivot_point(spec,tolerance,"gap-driven")
b = get_b_ion_series(pep)
y = get_y_ion_series(pep)
true_pivot = np.mean([*b[0:2],*y[-3:-1]])
print("predicted pivot",p)
print("true pivot",true_pivot)
print("error",abs(true_pivot - p))
print("predicted pivot symmetry, true pivot symmetry, expected symmetry",measure_mirror_symmetry(spec,p), measure_mirror_symmetry(spec,true_pivot), (len(spec) - 1)/len(spec))
overlap_pivots = [np.mean(x.data) for x in search(spec,"overlap",alphabet,tolerance)]
n = len(overlap_pivots)
for i in range(n):
    for j in range(i + 1, n):
        o1 = overlap_pivots[i]
        o2 = overlap_pivots[j]
        reconstructed_pivot = (o1 + o2) / 2
        if abs(measure_mirror_symmetry(spec, reconstructed_pivot) - (len(spec) - 1) / len(spec)) < 0.1:
            print(f"recovered pivot {i,j}: {o1} + {o2} = {reconstructed_pivot}")

In [ ]:
spec = generate_default_fragment_spectrum("QNPEVLTEYR")

In [ ]:
for ion, peak in zip (spec.getStringDataArrays()[0], spec):
    print(ion.decode(), peak.getMZ())

## Miscellaneous

#### Create Swiss-Prot Subset

In [ ]:
sprot = load_fasta_records("uniprot_sprot.fasta")

def create_sprot_subset(sprot,count,filename_pattern):
    shuffle(sprot)
    sprot_subset = sprot[:count]
    counter = 0
    while os.path.isfile(filename_pattern.format(counter)):
        counter += 1
    filename = filename_pattern.format(counter)
    print(filename)
    with open(filename,"w") as handle:
        print(SeqIO.write(sprot_subset, handle, "fasta"))

In [ ]:
create_sprot_subset(sprot,1000,"sprot_1k_{}.fasta")

In [ ]:
create_sprot_subset(sprot,10000,"sprot_10k_{}.fasta")

In [ ]:
create_sprot_subset(sprot,100000,"sprot_100k_{}.fasta")